In [1]:
from gudiff_model import PDBDataSet_GraphCon
from gudiff_model.Graph_UNet import GraphUNet
from data_rigid_diffuser.diffuser import FrameDiffNoise

from se3_transformer.model.fiber import Fiber
import torch
import os
import logging
from datetime import datetime
from collections import defaultdict
import time
import tree
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from torch import einsum
import numpy as np
import se3_diffuse.utils as du
import util.framediff_utils as fu
from data_rigid_diffuser import rigid_utils as ru
import copy
import util.pdb_writer 
from experiment.Experiment import Experiment
import json
import argparse




In [2]:
#mode meta

import pandas as pd

In [3]:
with open('configs/base_gun.json', 'r') as f:
            conf = json.load(f)
with open('configs/gun_cluster.json', 'r') as f:
            conf = json.load(f)
exp = Experiment(conf, name='t2', ckpt_model=None, ckpt_opt=None,swap_metadir=True)

In [4]:
conf['cluster_path']

'/mnt/h/datasets/procmmCIF/clusters-by-entity-30.txt'

In [5]:
exp.create_dataset()

(<torch.utils.data.dataloader.DataLoader at 0x7f89fc5d9c70>,
 <torch.utils.data.dataloader.DataLoader at 0x7f89fc5d9c70>)

In [4]:

cf = '/mnt/h/GUN_Saves/weights/GUN_10182024/step_1309440.pth'

In [4]:
# ckpt_model=torch.load(cf)['model']
# ckpt_opt = torch.load(cf)['optimizer']
# conf = torch.load(cf)['conf']
# conf['filter_dict']=True
# conf['meta_data_path'] = '/mnt/h/datasets/procmmCIF/metadata.csv'
# conf['num_heads']=8
# conf['batch_size']=8
# device='cuda'
# exp = Experiment(conf,name='bgd2', ckpt_model=ckpt_model, ckpt_opt=ckpt_opt,swap_metadir=True)

In [5]:
# exp.create_dataset()

(<torch.utils.data.dataloader.DataLoader at 0x7f7b187df040>,
 <torch.utils.data.dataloader.DataLoader at 0x7f7b187df040>)

In [6]:
# exp._model = exp._model.to(exp.device)

# (train_loader, valid_loader) = exp.create_dataset()


In [7]:
B=8

#visualize_T at selected t-values
vis_t = np.array([0.01,0.05,0.1,0.2,0.3,0.5,0.8,1.0])
vis_t = vis_t[None,...].repeat(int(np.ceil(B/len(vis_t))),axis=0).flatten()[:B]

index_in = exp.train_sample.generate_batch_indices(protein_length=128)
batch_feats = exp.generate_tbatch( index_in,vis_t)



batch_feats= tree.map_structure(
                lambda x: x.to(device), batch_feats)
noised_dict =   {'CA': batch_feats['CA_noised'] ,
                    'N_CA': batch_feats['N_CA_noised'].unsqueeze(-2) ,
                    'C_CA': batch_feats['C_CA_noised'].unsqueeze(-2) }


eval_dict = exp.eval_model(batch_feats,noised_dict)


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In [8]:
exp.train_sample.dataset    

In [9]:

sel_length = exp.train_sample._data_csv

In [13]:
sel_length['modeled_seq_len'].max()

409

In [11]:
exp.train_sample._data_csv['modeled_seq_len'].max()

409

In [43]:
generated = exp.reverse(65, write_out=True)

/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [42]:
generated

tensor([[[[ 2.0905,  0.1086, -1.5726],
          [ 1.9928,  0.0154, -1.5175],
          [ 1.9083, -0.0474, -1.6276]],

         [[ 1.7834, -0.2349, -1.6283],
          [ 1.7099, -0.3098, -1.7295],
          [ 1.5604, -0.2822, -1.7209]],

         [[ 1.5570, -0.2675, -1.6668],
          [ 1.4143, -0.2412, -1.6529],
          [ 1.3680, -0.1341, -1.7508]],

         ...,

         [[ 0.5418,  0.3370, -0.8856],
          [ 0.6369,  0.4458, -0.8662],
          [ 0.5659,  0.5803, -0.8576]],

         [[ 0.6918,  0.5949, -0.7654],
          [ 0.6595,  0.7366, -0.7541],
          [ 0.7240,  0.8171, -0.8661]],

         [[ 0.6080,  0.8244, -0.8951],
          [ 0.6489,  0.9155, -1.0013],
          [ 0.6568,  1.0590, -0.9510]]],


        [[[-0.9916,  0.1795, -2.9095],
          [-0.9796,  0.2374, -2.7762],
          [-1.0071,  0.1335, -2.6682]],

         [[-0.8821,  0.0678, -2.4834],
          [-0.9096, -0.0488, -2.4003],
          [-0.9049, -0.0134, -2.2522]],

         [[-1.0286, -0.1667, -2

In [14]:
eval_dir = '/home/nwoodall/GraphUNet_SE3Diff/Eval_Direc/test/'
epoch=240
util.pdb_writer.dump_tnp_epoch_direc(eval_dict['true'], 
                                            eval_dict['noise'], 
                                            eval_dict['pred'], vis_t, e=epoch, 
                                            numOut=len(vis_t), outdir=eval_dir)

In [14]:
eval_dict

{'true': array([[[[ 14.190878  ,  22.249401  ,   5.386326  ],
          [ 12.772044  ,  21.94571   ,   5.244343  ],
          [ 12.312856  ,  21.390211  ,   3.9027767 ]],
 
         [[ 11.33691   ,  20.412048  ,   3.894443  ],
          [ 10.679044  ,  19.979713  ,   2.6673434 ],
          [  9.9555645 ,  21.080399  ,   1.9029503 ]],
 
         [[  9.198129  ,  21.946674  ,   2.5895686 ],
          [  8.594045  ,  23.08471   ,   1.9073449 ],
          [  9.660548  ,  23.868069  ,   1.1535897 ]],
 
         ...,
 
         [[-15.058659  ,  15.664286  ,  -0.39225337],
          [-16.369955  ,  15.366711  ,  -0.95565605],
          [-16.700485  ,  13.887245  ,  -0.8101822 ]],
 
         [[-16.306559  ,  13.3636    ,   0.3573136 ],
          [-16.402954  ,  11.930714  ,   0.6083451 ],
          [-17.781197  ,  11.322078  ,   0.38641527]],
 
         [[-18.917887  ,  12.080217  ,   0.43481374],
          [-20.286957  ,  11.587711  ,   0.5273438 ],
          [-21.038855  ,  11.550717  ,  -0.

In [17]:
exp.start_training()

Using device: cuda
number of epochs 120
epoch 240
mem_used 419715072


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


KeyboardInterrupt: 

In [6]:
with open('configs/base_gun.json', 'r') as f:
            conf = json.load(f)
with open('configs/gun_cluster.json', 'r') as f:
            conf = json.load(f)
exp = Experiment(conf, name='t2', ckpt_model=None, ckpt_opt=None,swap_metadir=True)

In [7]:
a,b = exp.create_dataset()

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/procmmCIF/metadata.csv'

In [6]:
exp.dataset.csv

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration,index
116040,5i0c,/mnt/h/datasets/procmmCIF/i0/5i0c.pkl,/home/ubuntu/mmCIF/i0/5i0c.cif,1,monomeric,1.92,x-ray diffraction,1,homomer,155,89,0.280899,0.370787,0.348315,1.312202,0
163683,6qjj,/mnt/h/datasets/procmmCIF/qj/6qjj.pkl,/home/ubuntu/mmCIF/qj/6qjj.cif,1,monomeric,1.70,x-ray diffraction,1,homomer,112,89,0.393258,0.168539,0.438202,1.188336,1
140788,1cyi,/mnt/h/datasets/procmmCIF/cy/1cyi.pkl,/home/ubuntu/mmCIF/cy/1cyi.cif,1,monomeric,1.90,x-ray diffraction,1,homomer,158,89,0.382022,0.595506,0.022472,1.197820,2
146280,1hbk,/mnt/h/datasets/procmmCIF/hb/1hbk.pkl,/home/ubuntu/mmCIF/hb/1hbk.cif,1,monomeric,1.90,x-ray diffraction,1,homomer,169,89,0.370787,0.629213,0.000000,1.391454,3
56682,4htj,/mnt/h/datasets/procmmCIF/ht/4htj.pkl,/home/ubuntu/mmCIF/ht/4htj.cif,1,monomeric,2.00,x-ray diffraction,1,homomer,123,89,0.337079,0.393258,0.269663,1.219784,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162793,1jxy,/mnt/h/datasets/procmmCIF/jx/1jxy.pkl,/home/ubuntu/mmCIF/jx/1jxy.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.987290,644
162805,1jxx,/mnt/h/datasets/procmmCIF/jx/1jxx.pkl,/home/ubuntu/mmCIF/jx/1jxx.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.987236,645
162830,1jxw,/mnt/h/datasets/procmmCIF/jx/1jxw.pkl,/home/ubuntu/mmCIF/jx/1jxw.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.987437,646
166323,1ab1,/mnt/h/datasets/procmmCIF/ab/1ab1.pkl,/home/ubuntu/mmCIF/ab/1ab1.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.981559,647


In [7]:
exp.start_training()

Using device: cuda
number of epochs 120
epoch 0
mem_used 23944704


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


[1]: loss=0.2590, steps/sec=183.00979
0.25902265
eval Eval_Direc/t2_14D_10M_2024Y/14D_10M_2024Y_22h_56m_10s/step_2


/home/nwoodall/miniconda3/envs/se33/lib/python3.9/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


KeyboardInterrupt: 

In [3]:
ld = torch.load('/mnt/c/Users/nwood/OneDrive/Desktop/GraphUNet_SE3Diff/GUN_checkpoints/pdb200/08D_09M_2024Y_00h_20m_15s/step_17760.pth')

In [4]:
ld['conf']

{'batch_size': 4,
 'topk': 4,
 'stride': 4,
 'KNN': 30,
 'num_heads': 8,
 'channels': 32,
 'channels_div': 4,
 'nodefeats_0': 32,
 'nodefeats_1': 6,
 'num_layers': 1,
 'num_layers_ca': 2,
 'edge_feature_dim': 1,
 'latent_pool_type': 'avg',
 't_size': 12,
 'min_t': 0.001,
 'max_t': 1.0,
 'mult': 2,
 'zero_lin': True,
 'use_tdeg1': True,
 'cuda': True,
 'learning rate': 0.0005,
 'weight_decay': 5e-06,
 'device': 'cuda',
 'num_epoch': 120,
 'log_freq': 1000,
 'ckpt_freq': 5000,
 'early_chkpt': 2,
 'coord_scale': 10.0,
 'dataset_max': 5000,
 'meta_data_path': '/mnt/h/datasets/p200/metadata.csv',
 'sample_mode': 'length_batch',
 'ckpt_dir': 'GUN_checkpoints/',
 'eval_dir': 'Eval_Direc/',
 'trained_steps': 17760,
 'epoch': 17760}

In [3]:
fn = '/mnt/h/datasets/procMMCIF/metadata.csv'
dir_path = os.path.dirname(os.path.realpath(fn))
pdb_csv = pd.read_csv(fn)
pdb_csv['index'] = pdb_csv.index


In [4]:
filter_conf = {'allowed_oligomer': ['monomeric'],
                'max_loop_percent': 0.5}
pdb_csv = pdb_csv[pdb_csv.oligomeric_detail.isin(filter_conf['allowed_oligomer'])]
pdb_csv = pdb_csv[pdb_csv.coil_percent < filter_conf['max_loop_percent']]
pdb_csv = pdb_csv[pdb_csv.modeled_seq_len > 45]
pdb_csv = pdb_csv[pdb_csv.modeled_seq_len < 400]
pdb_csv = pdb_csv.sort_values('modeled_seq_len', ascending=False)

In [5]:
pdb_csv_batched = pdb_csv.copy()
sampled_order = pd.DataFrame(columns=pdb_csv.columns)

In [6]:
sampled_order

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration,index


In [7]:
pdb_csv_batched = pdb_csv.copy()
sampled_order = pd.DataFrame(columns=pdb_csv.columns)

pdb_csv_batched = pdb_csv.copy()
sampled_order = pd.DataFrame(columns=pdb_csv.columns)

while len(pdb_csv_batched)>1:
    pdb_csv_batched = pdb_csv_batched.loc[~pdb_csv_batched['index'].isin(sampled_order['index'])].copy()
    sampled_order_part = pdb_csv_batched.groupby('modeled_seq_len').sample(
                4, replace=True, random_state=0) #one batch per length
    sampled_order = pd.concat((sampled_order,sampled_order_part))


In [8]:
sampled_order

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration,index
162805,1jxx,/home/ubuntu/procmmCIF/jx/1jxx.pkl,/home/ubuntu/mmCIF/jx/1jxx.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.987236,162805
166323,1ab1,/home/ubuntu/procmmCIF/ab/1ab1.pkl,/home/ubuntu/mmCIF/ab/1ab1.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.981559,166323
55809,3u7t,/home/ubuntu/procmmCIF/u7/3u7t.pkl,/home/ubuntu/mmCIF/u7/3u7t.cif,1,monomeric,0.85,x-ray diffraction,1,homomer,97,46,0.456522,0.413043,0.130435,1.016466,55809
133219,1cnr,/home/ubuntu/procmmCIF/cn/1cnr.pkl,/home/ubuntu/mmCIF/cn/1cnr.cif,1,monomeric,1.05,x-ray diffraction,1,homomer,47,46,0.391304,0.478261,0.130435,0.982016,133219
18812,9evr,/home/ubuntu/procmmCIF/ev/9evr.pkl,/home/ubuntu/mmCIF/ev/9evr.cif,1,monomeric,2.49,x-ray diffraction,1,homomer,48,48,0.250000,0.750000,0.000000,1.026263,18812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166101,4wm4,/home/ubuntu/procmmCIF/wm/4wm4.pkl,/home/ubuntu/mmCIF/wm/4wm4.cif,1,monomeric,1.60,x-ray diffraction,1,homomer,298,129,0.465116,0.410853,0.124031,1.417568,166101
19570,3wun,/home/ubuntu/procmmCIF/wu/3wun.pkl,/home/ubuntu/mmCIF/wu/3wun.cif,1,monomeric,2.40,x-ray diffraction,1,homomer,163,129,0.488372,0.403101,0.108527,1.401967,19570
19570,3wun,/home/ubuntu/procmmCIF/wu/3wun.pkl,/home/ubuntu/mmCIF/wu/3wun.cif,1,monomeric,2.40,x-ray diffraction,1,homomer,163,129,0.488372,0.403101,0.108527,1.401967,19570
19570,3wun,/home/ubuntu/procmmCIF/wu/3wun.pkl,/home/ubuntu/mmCIF/wu/3wun.cif,1,monomeric,2.40,x-ray diffraction,1,homomer,163,129,0.488372,0.403101,0.108527,1.401967,19570


In [30]:
sampled_order

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration,index
162805,1jxx,/home/ubuntu/procmmCIF/jx/1jxx.pkl,/home/ubuntu/mmCIF/jx/1jxx.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.987236,162805
166323,1ab1,/home/ubuntu/procmmCIF/ab/1ab1.pkl,/home/ubuntu/mmCIF/ab/1ab1.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.981559,166323
55809,3u7t,/home/ubuntu/procmmCIF/u7/3u7t.pkl,/home/ubuntu/mmCIF/u7/3u7t.cif,1,monomeric,0.85,x-ray diffraction,1,homomer,97,46,0.456522,0.413043,0.130435,1.016466,55809
133219,1cnr,/home/ubuntu/procmmCIF/cn/1cnr.pkl,/home/ubuntu/mmCIF/cn/1cnr.cif,1,monomeric,1.05,x-ray diffraction,1,homomer,47,46,0.391304,0.478261,0.130435,0.982016,133219
12628,6w2i,/home/ubuntu/procmmCIF/w2/6w2i.pkl,/home/ubuntu/mmCIF/w2/6w2i.cif,1,monomeric,1.45,x-ray diffraction,1,homomer,97,48,0.312500,0.687500,0.000000,1.059563,12628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12062,8dep,/home/ubuntu/procmmCIF/de/8dep.pkl,/home/ubuntu/mmCIF/de/8dep.cif,1,monomeric,3.60,electron microscopy,1,homomer,402,398,0.190955,0.809045,0.000000,2.177890,12062
51574,1xqd,/home/ubuntu/procmmCIF/xq/1xqd.pkl,/home/ubuntu/mmCIF/xq/1xqd.cif,1,monomeric,1.80,x-ray diffraction,1,homomer,894,399,0.388471,0.513784,0.097744,2.120110,51574
160982,1gei,/home/ubuntu/procmmCIF/ge/1gei.pkl,/home/ubuntu/mmCIF/ge/1gei.cif,1,monomeric,1.60,x-ray diffraction,1,homomer,686,399,0.375940,0.511278,0.112782,2.116197,160982
109715,6n86,/home/ubuntu/procmmCIF/n8/6n86.pkl,/home/ubuntu/mmCIF/n8/6n86.cif,1,monomeric,3.90,x-ray diffraction,1,homomer,399,399,0.275689,0.714286,0.010025,2.630473,109715


In [18]:
pdb_depl = pdb_csv[~pdb_csv['index'].isin(sampled_order)]

In [19]:
pdb_depl.shape

(36700, 16)

In [63]:
pdb_csv['index'] = pdb_csv.index

In [64]:
for i in  iter(sampled_order['index'].tolist()):
    pdb_csv.loc[pdb_csv['index'] == i]

In [65]:
sampled_order

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration,index
133219,1cnr,/home/ubuntu/procmmCIF/cn/1cnr.pkl,/home/ubuntu/mmCIF/cn/1cnr.cif,1,monomeric,1.05,x-ray diffraction,1,homomer,47,46,0.391304,0.478261,0.130435,0.982016,133219
162830,1jxw,/home/ubuntu/procmmCIF/jx/1jxw.pkl,/home/ubuntu/mmCIF/jx/1jxw.cif,1,monomeric,0.89,x-ray diffraction,1,homomer,47,46,0.456522,0.413043,0.130435,0.987437,162830
42237,3v1a,/home/ubuntu/procmmCIF/v1/3v1a.pkl,/home/ubuntu/mmCIF/v1/3v1a.cif,1,monomeric,0.98,x-ray diffraction,1,homomer,100,48,0.229167,0.770833,0.000000,1.175839,42237
18812,9evr,/home/ubuntu/procmmCIF/ev/9evr.pkl,/home/ubuntu/mmCIF/ev/9evr.cif,1,monomeric,2.49,x-ray diffraction,1,homomer,48,48,0.250000,0.750000,0.000000,1.026263,18812
12618,6w2h,/home/ubuntu/procmmCIF/w2/6w2h.pkl,/home/ubuntu/mmCIF/w2/6w2h.cif,1,monomeric,1.60,x-ray diffraction,1,homomer,93,49,0.346939,0.653061,0.000000,1.066686,12618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54146,5nq7,/home/ubuntu/procmmCIF/nq/5nq7.pkl,/home/ubuntu/mmCIF/nq/5nq7.cif,1,monomeric,2.75,x-ray diffraction,4,heteromer,681,497,0.496982,0.124748,0.378270,2.220914,54146
16215,3gyj,/home/ubuntu/procmmCIF/gy/3gyj.pkl,/home/ubuntu/mmCIF/gy/3gyj.cif,1,monomeric,0.92,x-ray diffraction,1,homomer,1104,498,0.467871,0.291165,0.240964,2.220270,16215
98767,4oes,/home/ubuntu/procmmCIF/oe/4oes.pkl,/home/ubuntu/mmCIF/oe/4oes.cif,1,monomeric,1.95,x-ray diffraction,1,homomer,897,498,0.465863,0.315261,0.218876,2.388600,98767
69439,1n4w,/home/ubuntu/procmmCIF/n4/1n4w.pkl,/home/ubuntu/mmCIF/n4/1n4w.cif,1,monomeric,0.92,x-ray diffraction,1,homomer,1315,499,0.470942,0.292585,0.234469,2.222744,69439


In [ ]:
dirpath = '/gdrive_Test/'

In [4]:
fn = '/mnt/h/datasets/procmmCIF/metadata.csv'
dir_path = os.path.dirname(os.path.realpath(fn))
df = pd.read_csv(fn)

dirpath = '/gdrive_Test/'

In [4]:
os.path.abspath(os.path.join(os.path.dirname(fn)))

'/mnt/h/datasets/p200'

In [8]:
_dataset_indices = list(range(len(df)))
df['index'] = _dataset_indices

In [ ]:
chain_feats = self._process_csv_row(processed_file_path,index=idx)

In [9]:
sampled_order = df.groupby('modeled_seq_len').sample(8, replace=True, random_state=0) #one batch per length


In [17]:
sampled_order

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration,index
5826,4ttk,/mnt/h/datasets/p200/tt/4ttk.pkl,/mnt/h/datasets/mmCIF/tt/4ttk.cif,1,monomeric,1.25,x-ray diffraction,1,homomer,21,1,12.000,0.000,2.000,0.715255,5826
3967,6mw0,/mnt/h/datasets/p200/mw/6mw0.pkl,/mnt/h/datasets/mmCIF/mw/6mw0.cif,1,monomeric,0.78,x-ray diffraction,1,homomer,12,1,4.000,0.000,0.000,0.487701,3967
4701,6phq,/mnt/h/datasets/p200/ph/6phq.pkl,/mnt/h/datasets/mmCIF/ph/6phq.cif,1,monomeric,1.32,x-ray diffraction,1,homomer,46,1,3.000,23.000,0.000,1.174012,4701
4701,6phq,/mnt/h/datasets/p200/ph/6phq.pkl,/mnt/h/datasets/mmCIF/ph/6phq.cif,1,monomeric,1.32,x-ray diffraction,1,homomer,46,1,3.000,23.000,0.000,1.174012,4701
4701,6phq,/mnt/h/datasets/p200/ph/6phq.pkl,/mnt/h/datasets/mmCIF/ph/6phq.cif,1,monomeric,1.32,x-ray diffraction,1,homomer,46,1,3.000,23.000,0.000,1.174012,4701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,8buw,/mnt/h/datasets/p200/bu/8buw.pkl,/mnt/h/datasets/mmCIF/bu/8buw.cif,"2,2","dimeric,dimeric",2.80,x-ray diffraction,4,heteromer,200,200,0.480,0.135,0.385,1.865659,649
649,8buw,/mnt/h/datasets/p200/bu/8buw.pkl,/mnt/h/datasets/mmCIF/bu/8buw.cif,"2,2","dimeric,dimeric",2.80,x-ray diffraction,4,heteromer,200,200,0.480,0.135,0.385,1.865659,649
2701,1hvi,/mnt/h/datasets/p200/hv/1hvi.pkl,/mnt/h/datasets/mmCIF/hv/1hvi.cif,2,dimeric,1.80,x-ray diffraction,2,heteromer,200,200,0.395,0.040,0.555,1.729709,2701
2701,1hvi,/mnt/h/datasets/p200/hv/1hvi.pkl,/mnt/h/datasets/mmCIF/hv/1hvi.cif,2,dimeric,1.80,x-ray diffraction,2,heteromer,200,200,0.395,0.040,0.555,1.729709,2701


In [13]:
df.iloc[5826]

pdb_name                                            4ttk
processed_path          /mnt/h/datasets/p200/tt/4ttk.pkl
raw_path               /mnt/h/datasets/mmCIF/tt/4ttk.cif
oligomeric_count                                       1
oligomeric_detail                              monomeric
resolution                                          1.25
structure_method                       x-ray diffraction
num_chains                                             1
quaternary_category                              homomer
seq_len                                               21
modeled_seq_len                                        1
coil_percent                                        12.0
helix_percent                                        0.0
strand_percent                                       2.0
radius_gyration                                 0.715255
index                                               5826
Name: 5826, dtype: object

In [16]:
df.iloc[4701]

pdb_name                                            6phq
processed_path          /mnt/h/datasets/p200/ph/6phq.pkl
raw_path               /mnt/h/datasets/mmCIF/ph/6phq.cif
oligomeric_count                                       1
oligomeric_detail                              monomeric
resolution                                          1.32
structure_method                       x-ray diffraction
num_chains                                             1
quaternary_category                              homomer
seq_len                                               46
modeled_seq_len                                        1
coil_percent                                         3.0
helix_percent                                       23.0
strand_percent                                       0.0
radius_gyration                                 1.174012
index                                               4701
Name: 4701, dtype: object

In [10]:
sampled_order['index']

5826    5826
3967    3967
4701    4701
4701    4701
4701    4701
        ... 
649      649
649      649
2701    2701
2701    2701
5086    5086
Name: index, Length: 1592, dtype: int64

In [11]:
for x in iter(sampled_order['index'].tolist()):
    print(x)

5826
3967
4701
4701
4701
3969
4701
4697
3968
1248
1248
3968
1250
1249
1248
1249
5981
4429
5981
1733
4429
5980
5968
1733
5703
6334
6343
7010
7010
246
5124
5703
5151
3388
2106
2106
1974
4221
3723
6178
976
5400
5353
977
3279
6258
978
4437
1325
679
679
1731
4005
4005
4006
5962
4009
3805
2225
5778
6189
5629
638
638
6177
6266
6266
5667
5989
6177
5247
329
3408
4461
6459
6915
3825
6917
3825
5287
1725
1725
1724
1725
1725
4423
1725
1724
801
4382
4382
4111
4111
4111
243
711
3000
4783
1519
244
3000
244
724
749
2201
912
4534
1592
912
4022
4354
4534
3280
3187
4112
6456
196
4356
1732
4449
6576
5953
7234
1308
6526
1308
1318
7234
4485
3277
4485
4675
1367
3277
4485
1367
4130
2307
5399
4130
6734
4990
2096
4990
4263
4557
691
691
6482
691
6482
4557
2794
4924
100
6330
100
7101
100
7101
2325
2326
2325
6454
899
5406
5406
2326
2278
3855
3855
2278
5805
3901
5805
2278
4299
3412
5358
6188
6186
295
6186
4700
5039
164
5831
5039
4694
4620
6028
3171
2281
247
247
247
4699
4699
2281
4439
7102
7102
771
771
4070
4434
771

In [28]:
fn1 = df['processed_path'].iloc[0]

In [29]:
fn1

'/mnt/h/datasets/bCov_4H/00/0001581c72611c299a7297cf4c1d9851_0001.pkl'

In [31]:
os.path.basename(os.path.dirname(fn1))

'00'

In [35]:
df['processed_path'] = df['processed_path'].apply(lambda x: os.path.join(  dirpath,  os.path.basename((os.path.dirname(x))),  os.path.basename(x)) )

In [11]:
os.path.basename(x)

In [36]:
df['processed_path'].iloc[0]

'/gdrive_Test/00/0001581c72611c299a7297cf4c1d9851_0001.pkl'

In [15]:
df

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration
0,0001581c72611c299a7297cf4c1d9851_0001,/mnt/h/datasets/bCov_4H/00/0001581c72611c299a7...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.138462,0.861538,0.0,1.111881
1,0002182ba9e87d7b0d6b7d16b7001923_0001,/mnt/h/datasets/bCov_4H/00/0002182ba9e87d7b0d6...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.133914
2,00052a785cf4cffaa60b15d2ccc78f9d_0001_1,/mnt/h/datasets/bCov_4H/00/00052a785cf4cffaa60...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.111930
3,000701cc0c20e04627e1e44f84d5a6b6_0001_1,/mnt/h/datasets/bCov_4H/00/000701cc0c20e04627e...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.111956
4,000877380acb95e21726d9df052ec962_0001,/mnt/h/datasets/bCov_4H/00/000877380acb95e2172...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.104479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27889,fff532a1f3bd52993b00d58dc306e14c_0001_1,/mnt/h/datasets/bCov_4H/ff/fff532a1f3bd52993b0...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.090073
27890,fff8daf3a52878a3bf78d4aa4fd68612_0001_1,/mnt/h/datasets/bCov_4H/ff/fff8daf3a52878a3bf7...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.169231,0.830769,0.0,1.098638
27891,fffa17b132597274c1fd4fd217829317_0001_1,/mnt/h/datasets/bCov_4H/ff/fffa17b132597274c1f...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.111163
27892,fffaaf5372bf5d4f67462faa60fe4466_0001,/mnt/h/datasets/bCov_4H/ff/fffaaf5372bf5d4f674...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.099117


In [13]:
df['processed_path']

0        /gdrive_Test/0001581c72611c299a7297cf4c1d9851_...
1        /gdrive_Test/0002182ba9e87d7b0d6b7d16b7001923_...
2        /gdrive_Test/00052a785cf4cffaa60b15d2ccc78f9d_...
3        /gdrive_Test/000701cc0c20e04627e1e44f84d5a6b6_...
4        /gdrive_Test/000877380acb95e21726d9df052ec962_...
                               ...                        
27889    /gdrive_Test/fff532a1f3bd52993b00d58dc306e14c_...
27890    /gdrive_Test/fff8daf3a52878a3bf78d4aa4fd68612_...
27891    /gdrive_Test/fffa17b132597274c1fd4fd217829317_...
27892    /gdrive_Test/fffaaf5372bf5d4f67462faa60fe4466_...
27893    /gdrive_Test/fffab47f11ab9cbb43a0143b90b3e454_...
Name: processed_path, Length: 27894, dtype: object

In [ ]:
with open('configs/base_gun.json','r') as f:
    conf = json.load(f)



if __name__ == '__main__':
    parser = argparse.ArgumentParser('Train Graph U-Net')
    parser.add_argument('name', help='name of run',type=str)

    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument("-c", '--config_path',  help='path to config file in json format',type=str)
    group.add_argument("-k", '--checkpoint_file', help='previous checkpoint to load',type=str)

    args = parser.parse_args()

    if args.checkpoint_file is not None:

        ckpt_model=torch.load(args.checkpoint_file)['model']
        ckpt_opt = torch.load(args.checkpoint_file)['optimizer']
        conf = torch.load(args.checkpoint_file)['conf']

        exp = Experiment(conf,name=args.name, ckpt_model=ckpt_model, ckpt_opt=ckpt_opt)

    else:
        with open(args.config_path, 'r') as f:
            conf = json.load(f)
        exp = Experiment(conf, name=args.name, ckpt_model=None, ckpt_opt=None)

    tl, vl = exp.create_dataset() #load dataset from 'meta_data_path.csv'
    exp.start_training()